## Matriz de Transferencia: Método de Pochi-Yet

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
num_rep = 2
num = 2*num_rep
ni = 1 #índice de refracción del medio incidente
nf = 1 #índice de refracción del medio de salida
a_0 = 20 * np.pi / 180 # Para incidencias oblicuas
na = 3.0 #índice de refracción de la capa A
nb = 1.0 #índice de refracción de la capa B
a = 111.11 #Tamaño de la capa A, en micrómetros
b = 166.66 #Tamaño de la capa B, en micrómetros


def refractions(num,ni,nf,na,nb):
    '''
    Función para crear el arreglo de índices de refracción
    usamos las siguientes variables:
    Argumentos
    num = numero de repeticiones
    ni = indice de refracción del incidente
    nf = indice de refracción de salida
    na = indice de refracción de la capa a
    nb = indice de refracción de la capa b
    '''
    n = [ni]   
    for i in range(int(num/2)):
        n.append(na)
        n.append(nb)
    n.append(nf)
    return np.array(n)
    
def sizes(size_a,size_b,refraction_list):
    '''
    función para crear el arreglo de los tamaños de las capas
    Argumentos
    size_a = tamaño de la capa A
    size_b = tamaño de la capa B
    refraction_list = la lista con los índices de refracción, para usar como referencia
    '''
    d = refraction_list[1:-1]
    l = len(d)
    #nas = [size_a for i in range(l)]
    #nbs = [size_b for i in range(l)]
    n = []
    for i in range(int(num/2)):
        n.append(size_a)
        n.append(size_b)

    return np.array(n)

def positions(sizes):
    '''
    Función para crear el arreglo de las posiciones de cada capa
    Argumentos
    sizes = arreglo de los tamaños de las capas
    '''
    pos = [0]
    for i in range(len(sizes)):
        pos.append(pos[i] + sizes[i])
    return np.array(pos)



In [4]:
n = refractions(num,ni,nf,na,nb)
n

array([1., 3., 1., 3., 1., 1.])

In [5]:
d = sizes(a,b,n)
d

array([111.11, 166.66, 111.11, 166.66])

In [6]:
pos = positions(d)
pos

array([  0.  , 111.11, 277.77, 388.88, 555.54])

In [7]:
n

array([1., 3., 1., 3., 1., 1.])

Cálculo de los ángulos

In [8]:
def angles(a_0, refractions):
    '''
    función para el cálculo de los ángulos basado en los índices de refracción
    Argumentos:
    a_0 = ángulo inicial 
    refractions = arreglo que contiene los índices de refracción al interior del cristal
    '''
    a0 = a_0
    Ang = np.zeros(num+2)
    Ang[0] = a0
    for i in range(1,len(Ang)):
        Ang[i] = np.arcsin(refractions[i-1] / refractions[i] * np.sin(Ang[i-1]))

    
    return Ang[1:]


In [9]:
Ang = angles(a_0,n)
Ang

array([0.11425514, 0.34906585, 0.11425514, 0.34906585, 0.34906585])

Vectores de onda para cada capa

In [10]:
def k(Lambda,Angles,refractions):
    tabla = np.ones((len(Angles)))
    for i in range(len(tabla)):
        tabla[i] = tabla[i] * 2 * np.pi/Lambda * np.cos(Angles[i]) * refractions[i]        
    return tabla[:-1] 

def k_i(Lambda, a0, ni):
    return 2 * (np.pi/Lambda) * np.cos(a0) * ni

def k_f(Lambda, Angles, nf ):
    return 2 * (np.pi/Lambda) * np.cos(Angles[-1]) * nf


In [11]:
k(5,Ang,n)

array([1.24844376, 3.54255772, 1.24844376, 3.54255772])

Matrices dinámicas de las capas 

In [12]:
def dinamic_matrix(refractions, angles,num):
    '''
    función para calcular la matriz dinámica de las capas internas
    refractions = lista que contiene los indices de refracción de las capas
    angles = lista que contiene los angulos de refracción en las capas
    num = número de capas
    '''
    md = []
    for i in range(num):
        inner_list = [[1, 1], [-refractions[i] * np.cos(angles[i]), refractions[i] * np.cos(angles[i])]]
        md.append(inner_list)
    return np.array(md)    

def dinamic_matrix_i(a0,ni):
    mdi = [[1,1],[-ni*np.cos(a0), ni*np.cos(a0)]]
    return np.array(mdi)

def dinamic_matrix_f(nf,angles):
    mdf = [[1,1],[-nf* np.cos(angles[-1]), nf*np.cos(angles[-1])]]
    return np.array(mdf)

In [13]:
md = dinamic_matrix(n,Ang,num)
md

array([[[ 1.        ,  1.        ],
        [-0.99347998,  0.99347998]],

       [[ 1.        ,  1.        ],
        [-2.81907786,  2.81907786]],

       [[ 1.        ,  1.        ],
        [-0.99347998,  0.99347998]],

       [[ 1.        ,  1.        ],
        [-2.81907786,  2.81907786]]])

In [14]:
mdi = dinamic_matrix_i(a_0,ni)
mdi

array([[ 1.        ,  1.        ],
       [-0.93969262,  0.93969262]])

In [15]:
mdf = dinamic_matrix_f(nf,Ang)
mdf

array([[ 1.        ,  1.        ],
       [-0.93969262,  0.93969262]])

Matriz de Propagación

In [16]:
def propagation_matrix(w,Ang,distances):
    mp = []

# Calculate matrices based on the given formula
    for i in range(num):
        matrix = np.array([
            [np.exp(-1j * k(w,Ang,n)[i] * distances[i]), 0],
            [0, np.exp(1j * k(w,Ang,n)[i] * distances[i])]
        ])
        mp.append(matrix)

    return np.array(mp)

In [17]:
mp = propagation_matrix(5,Ang,d)
mp

array([[[0.88490347-0.46577446j, 0.        +0.j        ],
        [0.        +0.j        , 0.88490347+0.46577446j]],

       [[0.97660173+0.21505594j, 0.        +0.j        ],
        [0.        +0.j        , 0.97660173-0.21505594j]],

       [[0.88490347-0.46577446j, 0.        +0.j        ],
        [0.        +0.j        , 0.88490347+0.46577446j]],

       [[0.97660173+0.21505594j, 0.        +0.j        ],
        [0.        +0.j        , 0.97660173-0.21505594j]]])

In [18]:
def A(x, k, a, b):
    return np.exp(-1j * k(x,Ang,n)[0] * a) * np.cos(k(x,Ang,n)[1] * b) - 1j / 2 * np.sin(k(x,Ang,n)[1] * b) * (k(x,Ang,n)[1] / k(x,Ang,n)[0] + k(x,Ang,n)[0] / k(x,Ang,n)[1])




In [19]:
A(2,k,a,b)

(0.30142590567589816+0.020637085944767875j)